In [6]:
import ee
from datetime import datetime, timedelta
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
import matplotlib.pyplot as plt
datelist = pd.date_range(datetime.today(), periods=100).tolist()
from time import sleep
import dask.dataframe as dd
from dateutil.parser import parse
from dateutil.relativedelta import relativedelta
import os
import glob
from datetime import date

from uuid import uuid4


from shapely.geometry import Polygon, MultiPolygon, LineString, GeometryCollection
import plotly
import plotly.express as px
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import geopandas as gpd
import seaborn as sns

import dask.dataframe as dd
import geopandas as gpd
from heatwave_1 import detect
from datetime import timedelta
import seaborn as sns
from datetime import date
import matplotlib.pyplot as plt

sns.set_style('darkgrid')

In [7]:
#load dataframe of max temperatures for each county from 2016-2022
full_df = pd.read_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/daily_max/max_temps/max_temps.csv')
full_df['datetime'] = pd.to_datetime(full_df['datetime'])
full_df['temp_pct'] = full_df.groupby('location')['temperature_2m'].transform(lambda x: x.rank(pct=True))

# Identify heatwaves

In [8]:
#create dictionary containing each county's heatwave periods
heatwave_dic = {}
for county in full_df['location'].unique():
    county_df = full_df[(full_df['location'] == county)]
    county_df.reset_index(inplace=True)
    county_df.sort_values(by='datetime', inplace=True)
    county_df.drop_duplicates(['county', 'datetime'], inplace=True)
    
    #call 'detect' function from heatwave_1.py
    heatwave_dic[county] = detect(county_df['datetime_ord'].values, county_df['temperature_2m'].values, pctile=90, climatologyPeriod = [None, None], minDuration=2)


#create dictionary of dates for each heatwave
heatwave_dates = {}

#create dictionary of heatwave start day and lengths
consec_dates = {}

#generate unique ID code for each heatwave

heatwave_IDs = {}
for county in heatwave_dic.keys(): 
    consec_dates[county] = {}
    heatwave_dates[county] = []
    heatwave_IDs[county] = {}


    heatwave_starts = heatwave_dic[county][0]['date_start']
    heatwave_ends = heatwave_dic[county][0]['date_end']
    for i in range(len(heatwave_starts)):
        heatwave_code = str(uuid4())
        heatwave_days = pd.date_range(heatwave_starts[i],heatwave_ends[i],freq='d')
        length_heatwaves = len(heatwave_days)
        consec_dates[county][heatwave_days[0]] = length_heatwaves
        for item in heatwave_days:
            heatwave_dates[county].append(item)
            heatwave_IDs[county][item] = heatwave_code
            
            
heat_df = pd.DataFrame.from_dict(heatwave_dates, orient='index').unstack().reset_index()
heat_df.rename(columns={'level_1':'location', 0:'datetime'}, inplace=True)


heat_df.dropna(inplace=True)
heat_df['heatwave']=1

/var/folders/8g/qp1k_1g91m5f2m9pypxwxdhh0000gn/T/ipykernel_14605/2169280771.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  county_df.sort_values(by='datetime', inplace=True)
/var/folders/8g/qp1k_1g91m5f2m9pypxwxdhh0000gn/T/ipykernel_14605/2169280771.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  county_df.drop_duplicates(['county', 'datetime'], inplace=True)
/var/folders/8g/qp1k_1g91m5f2m9pypxwxdhh0000gn/T/ipykernel_14605/2169280771.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stabl

In [14]:
#merge full temperature dataset with identified 
full_df = full_df.merge(heat_df, on=['location', 'datetime'], how='left')

full_df['heatwave'] = full_df['heatwave'].replace(np.nan, 0)

In [15]:
full_df['pct'] = full_df.groupby('location')['temperature_2m'].transform(lambda x: x.rank(pct=True))

In [16]:
full_df[full_df['heatwave'] == 1].groupby('state')['temperature_2m'].describe()

,count,mean,std,min,25%,50%,75%,max
state,,,,,,,,
British Columbia,2994.0,26.823536,3.372774,17.726235,24.812239,26.934887,28.894093,40.556122
California,5671.0,36.132868,4.298267,24.768563,33.492891,35.891154,39.268191,48.168298
Oregon,3608.0,33.672870,2.841513,23.772982,32.317623,33.780383,35.370447,44.426642
Washington,4046.0,32.858483,4.403586,20.540545,29.814970,33.120834,35.761427,46.928690


In [17]:
#create dictionary of unique heatwave ID codes
codes = pd.DataFrame.from_dict(heatwave_IDs, orient='index').unstack().reset_index()

In [18]:
codes.columns=['datetime', 'location', 'heat_ID']

In [19]:
codes.dropna(inplace=True)

In [20]:
codes.head()

,datetime,location,heat_ID
0,2016-07-26,"Alameda, California",a69b15c4-450e-4689-a14d-cc9463d09b07
1,2016-07-26,"Alpine, California",2d6e7919-e1dd-4203-8ed7-e1f2eda92e51
2,2016-07-26,"Amador, California",f2e79367-41ce-44c6-9a2f-5fa147788080
3,2016-07-26,"Butte, California",880784aa-4acc-4e8b-b9b6-d84b4802e614
4,2016-07-26,"Calaveras, California",f95faca3-eab1-45bd-b965-70628055cae3


In [21]:
#get dataframe of heatwave durations
consec = pd.DataFrame.from_dict(consec_dates, orient='index').unstack().reset_index()

In [22]:
consec.columns= ['datetime', 'location', 'Heat Wave Duration']

In [23]:
consec['Heat Wave Duration'].max()

13.0

In [24]:
consec.head()

,datetime,location,Heat Wave Duration
0,2016-07-26,"Alameda, California",3.0
1,2016-07-26,"Amador, California",5.0
2,2016-07-26,"Butte, California",5.0
3,2016-07-26,"Colusa, California",5.0
4,2016-07-26,"Contra Costa, California",2.0


In [25]:
consec.dropna(inplace=True)

In [26]:
full_df = full_df.merge(codes, on=['datetime', 'location'])

In [27]:
full_df = full_df[full_df['year'] >= 2020]



In [28]:
full_df = full_df.merge(consec, on=['datetime', 'location'], how='left')

In [34]:
heatwaves = full_df.copy()

In [35]:
heatwaves['location'] = heatwaves['county'] + ', ' + heatwaves['state']

In [36]:
heatwaves['datetime'] = pd.to_datetime(heatwaves['datetime'])

In [37]:
heatwaves = heatwaves.merge(consec, on=['datetime', 'location'], how='left')

In [38]:
heatwaves = heatwaves[heatwaves['month'] != 10]

In [39]:
heatwaves.sort_values(by='datetime', inplace=True)

In [30]:
#save file
heatwaves.to_csv(f'/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/heatwave/heatwave_clean_90.csv')